# AlexNet — Architektúra

Súbor: alexnet_1.ipynb

Program: Hospodárska informatika

Vypracovala:  Veronika Motúzová 

Bakalárska práca: Metódy hlbokého učenia pre detekciu meteorov v astronomických snímkach

Vedúci bakalárskej práce: doc. Ing. Peter Butka, PhD.

Konzultant: Ing. Viera Maslej Krešňáková

Import knižníc

In [2]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.metrics import categorical_crossentropy
from keras.layers import Conv2D, MaxPooling2D, Dropout, ZeroPadding2D
from keras.utils import np_utils, plot_model
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
import array
from sklearn.metrics import confusion_matrix,classification_report
import itertools 
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.models import load_model 

Načítanie dát

In [3]:
train = np.load('../data/data_final_train.npz')
test = np.load('../data/data_final_test.npz')

In [4]:
x_train = train['train_data'].reshape(train['train_data'].shape[0], 128, 128, 1)
x_test = test['test_data'].reshape(test['test_data'].shape[0], 128, 128, 1)

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [6]:
print(x_train.shape)
print(x_test.shape)

(23746, 128, 128, 1)
(23746, 128, 128, 1)


In [7]:
y_train = np_utils.to_categorical(train['train_targets'], 2)
y_test = np_utils.to_categorical(test['test_targets'], 2)

CNN

In [8]:
model = Sequential()

In [9]:
model.add(ZeroPadding2D((1,1),input_shape=(128, 128,1)))

In [10]:
model.add(Conv2D(96, 3, strides=1, activation='relu')) # strides 4 -> 1 , 11 -> 3

In [11]:
model.add(MaxPooling2D(3, 2))

In [12]:
model.add(Conv2D(96, 5, strides=1, padding='same', activation='relu')) # 256 -> 96

In [13]:
model.add(MaxPooling2D(3, 2))

In [14]:
model.add(Conv2D(96, 3, strides=1, padding='same', activation='relu')) # 384 -> 96

In [15]:
model.add(Conv2D(96, 3, strides=1, padding='same', activation='relu'))

In [16]:
model.add(MaxPooling2D(3, 2))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(256, activation='relu')) # 4096 -> 256

In [19]:
model.add(Dropout(0.25)) # dropout 0.5 -> 0.25

In [20]:
model.add(Dense(256, activation='relu')) # 4096 -> 256

In [21]:
model.add(Dropout(0.25)) # dropout 0.5 -> 0.25

In [22]:
model.add(Dense(2, activation='softmax'))

Adam Optimization

In [23]:
adam = Adam()

In [24]:
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

Aby sa uložil model s najlepším výsledkom:

In [25]:
saved_model = "alexnet_1.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [26]:
f = model.fit(x_train, y_train, epochs=15, batch_size=128, validation_data=(x_test, y_test), callbacks=[checkpoint])


Train on 23746 samples, validate on 23746 samples
Epoch 1/15
23746/23746 [==============================] - 1813s 76ms/step - loss: 0.3889 - accuracy: 0.8169 - val_loss: 0.2209 - val_accuracy: 0.9197

Epoch 00001: val_accuracy improved from -inf to 0.91973, saving model to alexnet_1.hdf5
Epoch 2/15
23746/23746 [==============================] - 1792s 75ms/step - loss: 0.2099 - accuracy: 0.9229 - val_loss: 0.1847 - val_accuracy: 0.9312

Epoch 00002: val_accuracy improved from 0.91973 to 0.93123, saving model to alexnet_1.hdf5
Epoch 3/15
23746/23746 [==============================] - 3095s 130ms/step - loss: 0.1882 - accuracy: 0.9284 - val_loss: 0.1769 - val_accuracy: 0.9407

Epoch 00003: val_accuracy improved from 0.93123 to 0.94071, saving model to alexnet_1.hdf5
Epoch 4/15
23746/23746 [==============================] - 2514s 106ms/step - loss: 0.1487 - accuracy: 0.9473 - val_loss: 0.1089 - val_accuracy: 0.9657

Epoch 00004: val_accuracy improved from 0.94071 to 0.96572, saving model 

Načítanie modelu

In [35]:
print("Loading model....")
model = load_model('alexnet_1.hdf5')

Loading model....


Vyhodnotenie test loss a accuracy

In [36]:
#na trénovacej množine 
score = model.evaluate(x_train, y_train, verbose=0) 
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.017534747558914782
Test accuracy: 0.993556797504425


In [37]:
#na testovacej množine
score1 = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score1[0])
print('Test accuracy:', score1[1])

Test loss: 0.10167804184588745
Test accuracy: 0.9805862307548523


Predikcia

In [30]:
y_pred = model.predict(x_test)
prediction_int = np.zeros_like(y_pred)
prediction_int[y_pred > 0.5] = 1

Confusion Matrix

In [31]:
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))

In [32]:
print(matrix)
# TN / FN
# FP / TP

[[11795    78]
 [  383 11490]]


Classification Report

In [33]:
print(classification_report(y_test, prediction_int))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     11873
           1       0.99      0.97      0.98     11873

   micro avg       0.98      0.98      0.98     23746
   macro avg       0.98      0.98      0.98     23746
weighted avg       0.98      0.98      0.98     23746
 samples avg       0.98      0.98      0.98     23746



In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 96)      960       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 63, 63, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 96)        230496    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 96)        83040     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 31, 31, 96)       